<a href="https://colab.research.google.com/github/dSaif/EVA5/blob/main/EVA_S3X_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing Pytorch

!pip install torch
!pip install torchvision

In [ ]:
#Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
#Define Hyperparameters

num_classes = 62 # number of output classes discrete range [0,9]
num_epochs = 20 # number of times which the entire dataset is passed throughout the model
batch_size = 100 # the size of input data took for one iteration
lr = 1e-3 # size of step 

In [ ]:
#Downloading EMNIST data

train_data = dsets.EMNIST(root = './data', train = True,
                        transform = transforms.ToTensor(),
                        download = True,
                        split='byclass')

test_data = dsets.EMNIST(root = './data', train = False,
                       transform = transforms.ToTensor(),
                       split='byclass')

In [ ]:
#Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

In [ ]:
#Define model class

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.conv1= nn.Conv2d(in_channels= 1, out_channels= 10, kernel_size= 3, padding= 1) #28x28x1 > 28x28x10; RF: 3x3
    self.conv2= nn.Conv2d(in_channels= 10, out_channels= 10, kernel_size= 3,padding= 1 ) #28x28x10 > 28x28x10; RF: 5x5
    self.max_pool1= nn.MaxPool2d(2, stride= 2) #28x28x10 > 14x14x10 ; RF: 10x10
    self.conv3= nn.Conv2d(in_channels= 10, out_channels= 20, kernel_size= 3, padding=1 ) #14x14x10 > 14x14x20; RF: 12x12
    self.conv4= nn.Conv2d(in_channels= 20, out_channels= 20, kernel_size= 3, padding= 1) #14x14x20 > 14x14x20; RF: 14x14
    self.max_pool2= nn.MaxPool2d(2, stride= 2) #14x14x20 > 7x7x20 ; RF: 28x28
    self.conv5= nn.Conv2d(in_channels= 20, out_channels=30, kernel_size= 3 ) #7x7x20 > 5x5x30; RF: 30x30
    self.conv6= nn.Conv2d(in_channels= 30, out_channels= 30, kernel_size= 3) #5x5x30 > 3x3x30; RF: 32x32
    self.avg_pool= nn.AvgPool2d(3, stride= 2) # 3x3x30 > 1x1; RF: 34x34
    
    # self.fc1 = nn.Linear(30*3*3, 62)
    # self.relu = nn.ReLU()
    # self.fc2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self,t):
    t= self.max_pool1(F.relu(self.conv2(F.relu(self.conv1(t)))))
    t= self.max_pool2(F.relu(self.conv4(F.relu(self.conv3(t)))))
    t= self.conv6(F.relu(self.conv5(t)))
    t= t.reshape(-1, 30*3*3)
    out = self.avg_pool(t)
    # out = self.relu(out)
    # out = self.fc2(out)
    return t

In [ ]:
#Build the model
# !pip install torchsummary
from torchsummary import summary

net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  net.cuda()

summary(net, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
            Conv2d-2           [-1, 10, 28, 28]             910
         MaxPool2d-3           [-1, 10, 14, 14]               0
            Conv2d-4           [-1, 20, 14, 14]           1,820
            Conv2d-5           [-1, 20, 14, 14]           3,620
         MaxPool2d-6             [-1, 20, 7, 7]               0
            Conv2d-7             [-1, 30, 5, 5]           5,430
            Conv2d-8             [-1, 30, 3, 3]           8,130
            Linear-9                   [-1, 62]          16,802
Total params: 36,812
Trainable params: 36,812
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.21
Params size (MB): 0.14
Estimated Total Size (MB): 0.35
---------------------------------------------

In [ ]:
#Define loss-function & optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam( net.parameters(), lr=lr)

In [ ]:
#Training the model

for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    images = Variable(images).cuda()
    labels = Variable(labels).cuda()
    
    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))

Epoch [1/20], Step [100/6979], Loss: 3.8561
Epoch [1/20], Step [200/6979], Loss: 3.5252
Epoch [1/20], Step [300/6979], Loss: 3.0447
Epoch [1/20], Step [400/6979], Loss: 2.2377
Epoch [1/20], Step [500/6979], Loss: 2.0652
Epoch [1/20], Step [600/6979], Loss: 1.9581
Epoch [1/20], Step [700/6979], Loss: 1.4592
Epoch [1/20], Step [800/6979], Loss: 1.6453
Epoch [1/20], Step [900/6979], Loss: 1.5403
Epoch [1/20], Step [1000/6979], Loss: 1.9473
Epoch [1/20], Step [1100/6979], Loss: 1.4642
Epoch [1/20], Step [1200/6979], Loss: 1.0996
Epoch [1/20], Step [1300/6979], Loss: 1.2258
Epoch [1/20], Step [1400/6979], Loss: 1.5592
Epoch [1/20], Step [1500/6979], Loss: 0.9907
Epoch [1/20], Step [1600/6979], Loss: 1.1812
Epoch [1/20], Step [1700/6979], Loss: 1.3483
Epoch [1/20], Step [1800/6979], Loss: 1.0186
Epoch [1/20], Step [1900/6979], Loss: 1.1534
Epoch [1/20], Step [2000/6979], Loss: 1.0380
Epoch [1/20], Step [2100/6979], Loss: 1.1537
Epoch [1/20], Step [2200/6979], Loss: 1.0123
Epoch [1/20], Step 

In [ ]:
#Evaluating the accuracy of the model

correct = 0
total = 0
for images,labels in test_gen:
  images = Variable(images.view(-1,28*28)).cuda()
  labels = labels.cuda()
  
  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

RuntimeError: ignored